# 1. Preprocess-GroupImages
数据预处理：主要是将图片数据解压，然后复制到ImageDataGenerator需要的文件目录中。

## 导入包

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import zipfile
import pickle
from PIL import Image
import shutil

## 预览labels.csv

In [2]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

In [3]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...


In [4]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']

id_2_landmark_id_dict = dict(zip(train_id, train_landmark_id))
print('len(id_2_landmark_id_dict)=%d' % len(id_2_landmark_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))

len(id_2_landmark_id_dict)=1225029
id: cacf8152e2d2ae60, 	landmark_id:4676
id: 0a58358a2afd3e4e, 	landmark_id:6651


## 创建data_train，data_val，data_test文件夹和其对应的分类子文件夹

In [5]:
if not os.path.exists(train_folder):
    print('Create folder: %s' % train_folder)
    os.mkdir(train_folder)
else:
    print('Folder exists: %s' % train_folder)
    
if not os.path.exists(val_folder):
    print('Create folder: %s' % val_folder)
    os.mkdir(val_folder)
else:
    print('Folder exists: %s' % val_folder)

if not os.path.exists(test_folder):
    print('Create folder: %s' % test_folder)
    os.mkdir(test_folder)
else:
    print('Folder exists: %s' % test_folder)

if not os.path.exists(test_sub_folder):
    print('Create folder: %s' % test_sub_folder)
    os.mkdir(test_sub_folder)
else:
    print('Folder exists: %s' % test_sub_folder)

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_test
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_test/test


In [6]:
unique_landmark_ids = list(set(train_landmark_id))
print('len(unique_landmark_ids)=%d' % len(unique_landmark_ids))
sub_folder_name_len = len(str(len(unique_landmark_ids)))
print('sub_folder_name_len=%d' % sub_folder_name_len)


for c in unique_landmark_ids:
    train_sub_folder = os.path.join(train_folder, str(c).zfill(sub_folder_name_len))
    if not os.path.exists(train_sub_folder):
        print('Create folder: %s' % train_sub_folder)
        os.mkdir(train_sub_folder)
    else:
        print('Folder exists: %s' % train_sub_folder)

len(unique_landmark_ids)=14951
sub_folder_name_len=5
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00000
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00001
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00002
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00003
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00004
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00005
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00006
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00007
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00008
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00009
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00010
Create folder: /data1/kaggle/landm

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00535
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00536
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00537
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00538
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00539
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00540
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00541
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00542
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00543
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00544
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00545
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00546
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00983
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00984
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00985
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00986
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00987
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00988
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00989
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00990
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00991
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00992
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00993
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/00994
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01489
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01490
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01491
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01492
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01493
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01494
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01495
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01496
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01497
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01498
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01499
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/01500
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02107
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02108
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02109
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02110
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02111
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02112
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02113
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02114
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02115
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02116
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02117
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/02118
Crea

...

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12819
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12820
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12821
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12822
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12823
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12824
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12825
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12826
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12827
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12828
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12829
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/12830
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13456
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13457
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13458
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13459
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13460
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13461
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13462
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13463
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13464
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13465
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13466
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13467
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13723
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13724
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13725
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13726
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13727
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13728
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13729
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13730
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13731
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13732
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13733
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/13734
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14340
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14341
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14342
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14343
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14344
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14345
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14346
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14347
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14348
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14349
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14350
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14351
Crea

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14635
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14636
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14637
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14638
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14639
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14640
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14641
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14642
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14643
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14644
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14645
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_train/14646
Crea

In [7]:
for c in unique_landmark_ids:
    val_sub_folder = os.path.join(val_folder, str(c).zfill(sub_folder_name_len))
    if not os.path.exists(val_sub_folder):
        print('Create folder: %s' % val_sub_folder)
        os.mkdir(val_sub_folder)
    else:
        print('Folder exists: %s' % val_sub_folder)

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00000
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00001
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00002
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00003
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00004
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00005
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00006
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00007
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00008
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00009
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00010
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00011
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00405
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00406
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00407
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00408
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00409
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00410
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00411
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00412
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00413
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00414
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00415
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00416
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00745
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00746
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00747
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00748
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00749
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00750
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00751
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00752
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00753
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00754
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00755
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00756
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00932
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00933
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00934
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00935
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00936
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00937
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00938
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00939
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00940
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00941
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00942
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/00943
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01263
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01264
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01265
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01266
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01267
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01268
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01269
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01270
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01271
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01272
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01273
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/01274
Create folder: /data1/kaggle

...

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11393
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11394
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11395
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11396
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11397
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11398
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11399
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11400
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11401
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11402
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11403
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/11404
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12576
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12577
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12578
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12579
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12580
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12581
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12582
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12583
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12584
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12585
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12586
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12587
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12863
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12864
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12865
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12866
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12867
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12868
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12869
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12870
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12871
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12872
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12873
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/12874
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13556
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13557
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13558
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13559
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13560
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13561
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13562
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13563
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13564
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13565
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13566
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/13567
Create folder: /data1/kaggle

Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14554
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14555
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14556
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14557
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14558
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14559
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14560
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14561
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14562
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14563
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14564
Create folder: /data1/kaggle/landmark-recognition-challenge/input/data_val/14565
Create folder: /data1/kaggle

## 移动图片到data_train和data_test对应的子文件中

In [8]:
i = 0
train_valate = 0.02
data_train_amount = len(train_landmark_id)

def is_val(x, train_valate):
    n_val = train_valate * 100
    if x % 100 < n_val:
        return True
    return False


for i, image_name in enumerate(os.listdir(org_train_folder)):
    image_id = image_name[:-4]
    src_img = os.path.join(org_train_folder, image_name)
    if is_val(i, train_valate):
        target_img = os.path.join(val_folder, str(id_2_landmark_id_dict[image_id]).zfill(sub_folder_name_len), image_name)
    else:
        target_img = os.path.join(train_folder, str(id_2_landmark_id_dict[image_id]).zfill(sub_folder_name_len), image_name)
#     print(src_img)
#     print(target_img)
    if not os.path.exists(target_img):
        os.symlink(src_img, target_img)

    i = i+1
    if i%10000 == 0:
        print('%.2f' % (i/data_train_amount), end='  ')

0.01  0.02  0.02  0.03  0.04  0.05  0.06  0.07  0.07  0.08  0.09  0.10  0.11  0.11  0.12  0.13  0.14  0.15  0.16  0.16  0.17  0.18  0.19  0.20  0.20  0.21  0.22  0.23  0.24  0.24  0.25  0.26  0.27  0.28  0.29  0.29  0.30  0.31  0.32  0.33  0.33  0.34  0.35  0.36  0.37  0.38  0.38  0.39  0.40  0.41  0.42  0.42  0.43  0.44  0.45  0.46  0.47  0.47  0.48  0.49  0.50  0.51  0.51  0.52  0.53  0.54  0.55  0.56  0.56  0.57  0.58  0.59  0.60  0.60  0.61  0.62  0.63  0.64  0.64  0.65  0.66  0.67  0.68  0.69  0.69  0.70  0.71  0.72  0.73  0.73  0.74  0.75  0.76  0.77  0.78  0.78  0.79  0.80  0.81  0.82  0.82  0.83  0.84  0.85  0.86  0.87  0.87  0.88  0.89  0.90  0.91  0.91  0.92  0.93  0.94  0.95  0.96  0.96  0.97  0.98  0.99  

In [9]:
i = 0
data_test_amount = len(test_csv['id'])

for image_name in os.listdir(org_test_folder):
    src_img = os.path.join(org_test_folder, image_name)
    target_img = os.path.join(test_sub_folder, image_name)
#     print(src_img)
#     print(target_img)
    if not os.path.exists(target_img):
        os.symlink(src_img, target_img)

    i = i+1
    if i%1000 == 0:
        print('%.2f' % (i/data_test_amount), end='  ')

0.01  0.02  0.03  0.03  0.04  0.05  0.06  0.07  0.08  0.08  0.09  0.10  0.11  0.12  0.13  0.14  0.14  0.15  0.16  0.17  0.18  0.19  0.20  0.20  0.21  0.22  0.23  0.24  0.25  0.25  0.26  0.27  0.28  0.29  0.30  0.31  0.31  0.32  0.33  0.34  0.35  0.36  0.37  0.37  0.38  0.39  0.40  0.41  0.42  0.42  0.43  0.44  0.45  0.46  0.47  0.48  0.48  0.49  0.50  0.51  0.52  0.53  0.54  0.54  0.55  0.56  0.57  0.58  0.59  0.59  0.60  0.61  0.62  0.63  0.64  0.65  0.65  0.66  0.67  0.68  0.69  0.70  0.71  0.71  0.72  0.73  0.74  0.75  0.76  0.76  0.77  0.78  0.79  0.80  0.81  0.82  0.82  0.83  0.84  0.85  0.86  0.87  0.88  0.88  0.89  0.90  0.91  0.92  0.93  0.93  0.94  0.95  0.96  0.97  0.98  

In [10]:
print('Done!')

Done!
